# Assignment 3 

Modal synthesis, filters, spectral analysis and FM synthesis 
------------------------------------------------------------


In this assignment we will explore some fundamental DSP concepts 
to better understand the concepts we covered in class. In addition, 
we will look into the MIDI communication protocol and file format for 
controlling synthesizers and storing performance information. 

Similarly to the first assignment I will use the term familiar programming 
language to refer to the ones that you probably have encountered during your 
studies: Python, C, C++, Java, and Javascript. As you probably can guess I will use the
term unfamiliar programming language to refer to any other programming
language such as: Haskell, OCaml, Prolog, Rust, Go, Julia, Ruby, C#, F#
R, etc. I will use the term computer music textual languages to refer to languages 
that have extensive support and primitives for sound and music manipulation such as 
Chuck, Supercollider, Csound, Nyquist, and Faust and visual programming languages 
for languages such as PureData and Max/MSP. 

Unless explicitly stated you can use any programming languages for implementing 
the questions. Using a computer music language moves the degree of difficulty down and using an unfamiliar programming language moves it up. For example if you implement question 6 in Max/MSP or Chuck it counts as basic rather than expected. If you implement question 6 in an unfamiliar programming languages it can count as advanced. In general, I am flexible so if you want to adjust things just let me know. Also if you think of a question of comparable difficulty that you would like do again let me know and most likely it should be ok. 

If you need access to devices ask me via email or through Discord for access to ECS602. 

In [1]:
import IPython.display as ipd
import numpy as np
import bokeh 
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
from scipy.io import wavfile
import scipy.signal 

output_notebook()

Loading BokehJS ...

1. (Basic) Watch one of the videos listed under [Resources](resources.md). Select a short part of the video (something like 15-30 seconds) and make an explicit connection to any of the concepts we have covered in the course. Write a short paragraph describing the connection and provide the timing of the excerpt you considered. 


<span style="color:red">I watched the "Thomas Dolby synthesizer explanation" video. It has an excellent explanation of an oscillator and a filter (1:40-1:55, 2:20-3:40), which I also have learned in the course. The video explained the oscillator controled the pitch of the sound, and the filter controlled the tone of the sound. Very short and concise. And I like a fly in a match box analogy.</span>

2. (Basic) Watch one of the interviews listed under [Resources](resources.md). Select a short part of the interview (something like 30-60 seconds) and make an explicit connection to any of the concepts we have covered in the course. Write a short paragraph describing the connection and provide the timing of the excerpt you considered. 

<span style="color:red">I watched the interview with Perry Cook. There was a part he mentioned Chowning mistyped the frequency and got a complex sound (17:30-24:00). This accidental discovery was also discussed in class and I found it interesting. Then it was named frequency modulation, which I learned in the course. </span>

3. (Basic) Get two different coffee mugs and record the sound of hitting them with a pen. Plot the corresponding magnitude spectra (there is some code for plotting magnitude spectra in Python in the FM synthesis notebook) - you are welcome to use other tools. 

In [2]:
mug1sr, mug1 = wavfile.read('./audio/mug1.wav')
mug2sr, mug2 = wavfile.read('./audio/mug2.wav')

/tmp/ipykernel_36246/3549385007.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  mug2sr, mug2 = wavfile.read('./audio/mug2.wav')


In [3]:
ipd.Audio(mug1,rate=mug1sr)

In [4]:
ipd.Audio(mug2,rate=mug2sr)

In [24]:
def plot_mag_spectra(output, srate, title):
    mag_spectrum = abs(np.fft.rfft(output))
    p = figure()
    freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
    max_freq_bin = int(srate / len(mag_spectrum) * 5000)
    p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
    p.title.text_font_size = '20pt'
    p.title.text = title
    show(p)

In [6]:
plot_mag_spectra(mug1, mug1sr, "Mug 1")

In [7]:
plot_mag_spectra(mug2, mug2sr, "Mug 2")

4. (Basic) Create two additive synthesis models of the coffee mug recordings from question 3. Each model should consist of 4 sinusoidal oscillators and associated envelopes. Listen to the resulting sound and comment on whether you can recognize which coffee mug is which from the additive synthesis approximation. You can use any language (including computer music languages) for this question. 

In [8]:
# From lecture notebook
def plot(data_list): 
    fig, ax = plt.subplots(figsize=(4,3))
    for data in data_list: 
        plt.plot(data)    

def envelope(segments,srate,duration): 
    nsamples = int(srate*duration)
    value = 0.0
    segment_index = 0 
    data = np.zeros(nsamples)
    segment_sample = 0 
    prev_target = 0.0

    for i in np.arange(nsamples): 
        if (segment_index < len(segments)): 
            target = segments[segment_index][0]
            ramp_time = segments[segment_index][1]
            delay_time = segments[segment_index][2]
            
            ramp_samples = (ramp_time / 1000.0) * srate 
            delay_samples = (delay_time / 1000.0) * srate
            
            if i < segment_sample + ramp_samples: 
                incr = (target-prev_target) / ramp_samples 
            elif i < segment_sample + ramp_samples + delay_samples: 
                incr = 0.0 
            else: 
                if ramp_samples != 0.0: 
                    incr = (target-prev_target) / ramp_samples 
                else: 
                    incr = 0.0 
                segment_sample = i 
                segment_index = segment_index+1 
                prev_target = target 
            value = value + incr 
        data[i] = value
    return data  

def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data

In [9]:
def synth_model(srate, dur, f1, f2, f3, f4, s1, s2, s3, s4):
    penv1 = envelope(s1, srate, dur)
    penv2 = envelope(s2, srate, dur)
    penv3 = envelope(s3, srate, dur)
    penv4 = envelope(s4, srate, dur)
    osc1 = sinusoid(f1, dur=dur, srate=srate)
    osc2 = sinusoid(f2, dur=dur, srate=srate)
    osc3 = sinusoid(f3, dur=dur, srate=srate)
    osc4 = sinusoid(f4, dur=dur, srate=srate)
    return 0.25*(penv1 * osc1 + penv2 * osc2 + penv3 * osc3 + penv4 * osc4)

In [10]:
s1 = [(0.8, 10, 10), (0.35, 20, 17), (0.2, 37, 10), (0.1, 47, 20)]
s2 = [(0.9, 12, 10), (0.35, 22, 15), (0.3, 37, 10), (0.2, 47, 15)]
s3 = [(0.8, 25, 12), (0.45, 37, 20), (0.15, 57, 20), (0.1, 77, 10)]
s4 = [(0.9, 15, 15), (0.35, 30, 10), (0.3, 40, 15), (0.2, 55, 15)]

dur = 0.25
addsynth1 = synth_model(mug1sr, dur, 1000, 450, 230, 800, s1, s2, s3, s4)
ipd.Audio(addsynth1,rate=mug1sr)

In [11]:
s1 = [(0.8, 10, 10), (0.35, 20, 17), (0.2, 37, 10), (0.1, 47, 20)]
s2 = [(0.9, 12, 10), (0.35, 22, 15), (0.3, 37, 10), (0.2, 47, 15)]
s3 = [(0.8, 25, 12), (0.45, 37, 20), (0.15, 57, 20), (0.1, 77, 20)]
s4 = [(0.9, 15, 15), (0.35, 30, 10), (0.3, 40, 15), (0.2, 55, 15)]

dur = 0.25
addsynth2 = synth_model(mug2sr, dur, 11040, 5361, 5350, 2579, s1, s2, s3, s4)
ipd.Audio(addsynth2,rate=mug2sr)

<span style="color:red">Yes, I can recognize which mug is which based on the high or low frequency, but I cannot recognize the mug sounds. This sounds more like a musical instrument.</span>

5. (Expected) Create two modal synthesis models of the coffee mug recordings from question 3. Each model should consist of 4 BiQuad filters with appropriate associated center frequencies and resonances excited by an impulse function. Listen to the resulting sound and comment on whether you can recognize which coffee mug is which from the modal synthesis approximation. You can use any language (including computer music languages) for this question. 

In [12]:
# Lecture code
def band_pass(audio, freq, q, srate): 
    b = np.zeros(3)
    a = np.zeros(3)
    # center frequency in radians 
    frad = 2 * np.pi * freq / srate 

    # bandpass formulas 
    alpha_ = np.sin(frad)/(2*q)
    b[0] = np.sin(frad)/2
    b[1] =  0
    b[2] = -np.sin(frad)/2
    a[0] = 1 + alpha_
    a[1] = -2 * np.cos(frad) 
    a[2] = 1 - alpha_
    
    # apply filter once
    filtered_audio = scipy.signal.lfilter(b, a, audio)
    return filtered_audio 

In [13]:
srate = mug1sr

impulse = np.zeros(srate)
impulse[int(srate/2)] = 1
modelsynth1 = band_pass(impulse, 800, 100, srate)
ipd.Audio(modelsynth1, rate = srate)

In [14]:
srate = mug2sr

impulse = np.zeros(srate)
impulse[int(srate/2)] = 1
modelsynth2 = band_pass(impulse, 5000, 300, srate)
ipd.Audio(modelsynth2, rate = srate)

<span style="color:red">Yes, I can recognize which mug is which based on the high or low frequency, but I cannot recognize the mug sounds. I adjust the center frequency based on the magnitude spectra from question 3, but the sound is not as rich as the original.</span>

6. (Expected) Compare the mangitude spectra of the original mug recordings, the additive synthesis approximation, and the modal synthesis approximation. Describe what you observe and comment on whether it corresponds to what you hear. 

In [15]:
plot_mag_spectra(addsynth1, mug1sr, "Additive Synth Mug 1")

In [16]:
plot_mag_spectra(addsynth2, mug2sr, "Additive Synth Mug 2")

In [17]:
plot_mag_spectra(modelsynth1, mug1sr, "Model Synth Mug 1")

In [18]:
plot_mag_spectra(modelsynth2, mug2sr, "Model Synth Mug 2")

<span style="color:red">The magnitude spectra of the original mug recordings (question 3), the additive synthesis approximation, and the modal synthesis approximation all have peaks focusing around the same frequency ranges. However, the original ones have much more peaks with different heights, which means they have rich sounds. The additive synthesis plots only have 4 peaks, so the sounds are not as rich. The model synthesis plots have 1 peaks but they depict the mug sound better than the additive synthesis ones. This may be because the mug sound has the same behaviour as an impulse.</span>

7. (Expected) Synthesize a percussive sound using FM synthesis. The attack should be short and there should be many frequencies (i.e a high modulation index). Experiment with the amplitude and modulation index envelope shape until you have a good percussive sound. 

In [19]:
# Lecture code
def hz2radians(f, srate):
    return 2 * np.pi * f / srate

def frequency_modulation(start, end, freq, amp, mc_ratio, index, srate,env): 
    output = np.zeros(end-start)
    carrier_phase = 0.0 
    carrier_phase_incr = hz2radians(freq,srate)
    modulator_phase_incr = hz2radians(mc_ratio * freq,srate)
    
    amp_env = env  
    # get centered sin after integration 
    modulator_phase = 0.5 * (np.pi + modulator_phase_incr) 
    fm_index = hz2radians((mc_ratio * freq * index), srate)
    
    ind_env = fm_index * env
    
    for t in np.arange(start, end): 
        modulation = ind_env[t] * np.sin(modulator_phase)
        output[t] = env[t] * np.sin(carrier_phase) 
        carrier_phase += (modulation + carrier_phase_incr)
        modulator_phase += modulator_phase_incr
    return output 

In [20]:
srate = 44000
s1 = [(1, 200, 0), (0.6,100,0), (0.5, 500,0), (0.0, 100, 0)]
env = envelope(s1, srate, 1)
output = frequency_modulation(0, srate, 500, .9, 0.3, 4, srate, env)
ipd.Audio(output, rate=srate)

8. (Expected) Implement the parametric two pole filter used in the water bottle modal synthesis paper and described in this publication - create two audio examples showing how it can be used: https://ccrma.stanford.edu/~jos/smac03maxjos/smac03maxjos.pdf

In [20]:
# I don't entirely follow the paper
def two_pole_filter(audio, fs, freqs, taus, real_amp, img_amp):
    n = len(freqs)
    x = [0] * len(audio)
    y = [0] * len(audio)
    
    for i in range(len(audio) - 1):
        denom = np.maximum(0.00001, taus[i % n] * fs)
        exponent = -1/denom
        
        # I add amplitude to see if the loud and low effects would change anything
        x1 = np.exp(exponent) * img_amp[i % n] * np.cos(2 * np.pi * freqs[i % n]/fs)
        y1 = np.exp(exponent) * real_amp[i % n] * np.sin(2 * np.pi * freqs[i % n]/fs)
    
        x[i + 1] = x1 * x[i] - y1 * y[i] + audio[i]
        y[i + 1] = y1 * x[i] + x1 * y[i]
    
    return np.array(x) + np.array(y)

In [5]:
# freq, tau, amp values are taken from here https://github.com/jatinchowdhury18/modal-waterbottles/blob/master/WaterbottleSynth/Source/ModalVoice.cpp
freqs = [0.0, 735.0, 927.3, 977.0, 1371.7, 1739.3, 1782.3,
         1928.7, 2151.0, 2182.7, 2206.0, 2591.3, 2733.3, 3359.3,
         3461.7, 3678.0, 3751.0, 3977.0, 4192.0, 4278.7, 4601.0, 4796.0,
         5216.0, 5319.0, 5653.3, 5962.3, 6056.7, 6955.3, 7344.7, 7429.0,
         7599.0, 7761.3, 7945.0, 8384.3, 8522.7, 8772.0, 9128.3, 9833.7, 10376, 11209]
taus = [0.0, 0.0, 21474.8, 21694.8, 14806.6, 13126.2, 13010.7, 12464.7,
        11472.7, 11345.0, 11250.4, 8989.90, 8293.51, 6026.65, 5731.55,
        5402.86, 5340.44, 5162.93, 5026.59, 4957.85, 4804.45, 4752.26,
        4595.37, 4567.85, 4616.46, 4422.07, 4375.13, 3496.75, 3281.70,
        3239.40, 3170.67, 3117.73, 3065.84, 2912.81, 2880.41, 2808.99,
        2726.79, 2531.38, 2357.52, 2091.17]
real_amp = [1.6375e-4, 2.4356e-3, 1.4482e-3, 5.4375e-4, -9.101e-4,
            3.3461e-3, 9.2404e-4, -9.274e-4, -1.429e-3, -6.974e-4,
            -8.597e-4, 3.2111e-4, -1.467e-3, 1.1023e-3, -3.712e-4,
            -1.659e-3, -8.134e-4, 9.9300e-4, 1.2352e-3, 1.0866e-3,
            -1.961e-3, 2.4649e-3, -1.754e-3, -3.785e-3, -1.370e-3,
            -2.485e-4, 3.7174e-5, 8.5018e-4, -3.269e-3, 1.0449e-3,
            -1.951e-4, 9.5586e-4, -1.841e-3, 4.1932e-4, 2.4407e-3,
            -1.047e-3, -1.937e-3, 1.7622e-4, 1.9720e-3, 2.3114e-3]
img_amp = [8.2776e-4, -1.492e-3, 1.9481e-3, 2.6515e-3, -1.718e-3,
           2.8032e-3, 1.6469e-3, 9.0368e-4, -2.004e-3, -9.022e-4,
           -1.253e-3, 1.0081e-3, 1.4958e-3, 1.1865e-3, 3.3302e-3,
           1.7872e-4, -2.297e-4, -1.626e-3, 8.9491e-4, 3.1236e-3,
           -1.558e-3, -2.387e-3, 7.7755e-4, 2.7729e-4, 6.7944e-4,
           1.2892e-3, 1.3199e-3, 1.0479e-3, 8.0851e-4, 8.6837e-4,
           3.7306e-3, 3.3230e-5, -1.429e-3, 2.4528e-3, -2.483e-3,
           5.7032e-4, 1.3581e-3, 1.6890e-3, 1.2535e-4, 2.8051e-5]

In [165]:
fs = 44100
noise = np.random.normal(0, 1.0, fs)
noise_out = two_pole_filter(noise, fs, freqs, taus, real_amp, img_amp)
ipd.Audio(noise_out, rate=fs)

In [167]:
fs = 44100
t = np.linspace(0,1,fs)
sinusoid = np.sin(2 * np.pi * 440 * t)
sin_out = two_pole_filter(sinusoid, fs, freqs, taus, real_amp, img_amp)
ipd.Audio(sin_out, rate=fs)

<span style="color:red">It didn't sound like water bottle but I tried :( I read the Forced Solution in the paper. I'm not sure how they got the frequency, tau, and whatnot. I read their code and got the values there https://github.com/jatinchowdhury18/modal-waterbottles/blob/master/WaterbottleSynth/Source/ModalVoice.cpp</span>

9. (Advanced) Use the parametric two-pole filter mentioned in question 8 to create another modal synthesis approximation for the two coffee mugs of questions 5,6. Repeat the comparison between the different models both in terms of magnitude spectrums and in terms of listening. The water bottle modal synthesis paper (https://dafx2020.mdw.ac.at/proceedings/papers/DAFx2020_paper_24.pdf) describes a process for estimating the mode decay that you can follow for the coffee mug recordings. 



In [17]:
def mug_filter(audio, fs, mugsound=mug2):
    mag_spectrum = abs(np.fft.rfft(mugsound))
    freqs = np.linspace(0, 0.5 * fs, len(mag_spectrum))
    freqs[freqs < 70] = 0 # clean up mug frequencies a bit
    
    n = len(taus)
    x = [0] * len(audio)
    y = [0] * len(audio)
    
    for i in range(len(audio) - 1):
        denom = np.maximum(0.00001, taus[i % n] * fs)
        exponent = -1/denom
        
        # I add amplitude to see if the loud and low effects would change anything
        x1 = np.exp(exponent) * img_amp[i % n] * np.cos(2 * np.pi * freqs[i % len(freqs)]/fs)
        y1 = np.exp(exponent) * real_amp[i % n] * np.sin(2 * np.pi * freqs[i % len(freqs)]/fs)
    
        x[i + 1] = x1 * x[i] - y1 * y[i] + audio[i]
        y[i + 1] = y1 * x[i] + x1 * y[i]
    
    return np.array(x) + np.array(y)

In [28]:
fs = mug1sr
t = np.linspace(0,1,fs)
sinusoid = np.sin(2 * np.pi * 440 * t)
mug1_sin = mug_filter(sinusoid, fs, mug1)
ipd.Audio(mug1_sin, rate=fs)

In [29]:
plot_mag_spectra(mug1_sin[:100], fs, "2 pole filter for mug 1")

In [30]:
fs = mug2sr
t = np.linspace(0,1,fs)
sinusoid = np.sin(2 * np.pi * 440 * t)
mug2_sin = mug_filter(sinusoid, fs, mug2)
ipd.Audio(mug2_sin, rate=fs)

In [31]:
plot_mag_spectra(mug2_sin[:100], fs, "2 pole filter for mug 2")

<span style="color:red">It doesn't sound like the mug sound. I understand that the decay mode is similar to the release part in ADSR, and the way they find it is to do linear regression to find a line that fits the points on the time and magnitude (dB) plot. However, I don't know how to calculate the decay rate, so I get the values from the authors implementation https://github.com/jatinchowdhury18/modal-waterbottles/blob/master/WaterbottleSynth/Source/ModalVoice.cpp</span>

10. (Advanced) Implemented a real-time version of any of the synthesis models you developed in this assignments. You can use any language/programming environment for this question. 

In [26]:
# Assume that the input always need only 1 impulse
def real_time_model(audio, srate):
    mag_spectrum = abs(np.fft.rfft(audio))
    freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
    freqs[freqs < 70] = 0 # clean up audio a bit
    freq = np.average(freqs)/2
    
    print(freq)
    
    impulse = np.zeros(srate)
    impulse[int(srate/2)] = 1
    model = band_pass(impulse, freq, 100, srate)
    return model

In [27]:
model = real_time_model(mug1, mug1sr)
ipd.Audio(model, rate = mug1sr)

1999.848726885468
